In [1]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np

# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [2]:
with open('SP-groupedByDays.pkl', 'rb') as f:
    spDaily = pickle.load(f)

In [3]:
with open('spydf.pickle', 'rb') as f:
    tickData = pickle.load(f)

In [4]:
import random
random.shuffle(spDaily)

In [5]:
trainLen = int(len(spDaily)*0.75)

In [6]:
trainData=spDaily[:trainLen]
testData=spDaily[trainLen:]

In [7]:
tempData=spDaily[:1]

# Creating the Algorithim

In [8]:
algoTools = AlgoTools(tickData)

In [9]:
def Algo(data,r, a_L, a_S, E_e, stopProf, stopLoss):
    trader = Trader()
    
    #for each day in the data
    for d in range(len(data)):
        #for each time stamp in the day
        index = 1
        print("Day",d)
        for t in range(data[d].index[2] , data[d].index[len(data[d])-1]):
            index = index + 1
            
            #close out because market is closing                
            ti = tickData.loc[t,"Time2"].hour*100 + tickData.loc[t,"Time2"].minute
            if (1513<ti<1520 or 2354<ti<2359):
                if trader.getLastTradePos() != 0:
                    trader.closeOut(data[d].iloc[index,:])
                continue
            
            


            
            #if the acceleration is greater than or equal to acceleration constant Long
            if (algoTools.getAccel(t) >= a_L or algoTools.getRate(t) <= r) and trader.getLastTradePos() == 0:
                trader.goLong(data[d].iloc[index,:])
                continue
            
            #same as previous statment but for shorting
            if  ( algoTools.getAccel(t) <= a_S or algoTools.getRate(t) <= r ) and trader.getLastTradePos() == 0:
                trader.goShort(data[d].iloc[index,:])
                continue
            
            
            
            
           
            
            #-----Closing Conditions------
            
            
            #as per calclus reasoing if the first deravitive is = 0 it has found a local extrema. therfore close out 
            #E_e is error of extrema
            if algoTools.getRate(t) <= E_e and trader.getLastTradePos() != 0:
                trader.closeOut(data[d].iloc[index,:])
                continue
               
            
            #stop profit (in points)
            if (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() >= stopProf ) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] >= stopProf):
                trader.closeOut(data[d].iloc[index,:])
                continue
            
            #stop loss (in points)
            if (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() <= stopLoss) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] <= stopLoss ):
                trader.closeOut(data[d].iloc[index,:])
                continue
                
            
            
                
            
    return trader

In [10]:
trader = Algo(tempData,0.3, 0.6, -0.5, 0.03 ,0.5, -0.05)

Day 0


In [11]:
trader.getLR()

0    0.487941
dtype: float64

In [12]:
trader.getTradeInfo()

{'Total Return': 255.00000000006366,
 'Max Return': 29.999999999995453,
 'Min Return': -20.000000000004547}